In [ ]:
from repo_checker import *
from issue_formater import *
import pandas as pd

In [ ]:
challenges_data = load_challenges("retos_data.json")

In [ ]:
challenges_data

In [ ]:
df_repos = pd.read_csv("data/students.csv")

In [ ]:
for index, row in df_repos.iterrows():
    owner, repo, url = format_url(row["url"])
    print(f"Reviewing: {owner}/{repo}")
    for challenge in challenges_data:
        issue_title = challenge["title"]
        file_paths = challenge["paths"]
        max_date = challenge["max_date"]
        files_status = []
        dates_review = []
        if not check_issue_exists(url, issue_title):
            for path in file_paths:
                status = check_file_exists(url, path)
                files_status.append(status)
                if status:
                    dates_review.append(check_commit_dates(url, path, max_date))
                else:
                    dates_review.append(False)
            issue_body = summary_preview(files_status, dates_review, challenge)
            create_issue(url,owner, repo, issue_body, issue_title)